## k means clusterization

In [1]:
import os
import cv2
import csv
import math
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

## Sepeartion

In [2]:
#For Infected

#Taking all names
ParasitizedFilename = os.listdir("path/DataSet/Parasitized")
# print(len(ParasitizedFilename))
#Sorting array by name
ParasitizedFilename = sorted(ParasitizedFilename)#Files total=13799

#For Uninfectedimport csv

#Taking all names
UninfectedFilename = os.listdir("path/DataSet/Uninfected")   
#Sorting array by name
UninfectedFilename = sorted(UninfectedFilename)#Files total=13799
print("Separtion is done")

Sepeartion is done


In [3]:
# #csv of Filenames
# pd.DataFrame(ParasitizedFilename).to_csv("ParasitizedFilename1.csv")
# pd.DataFrame(UninfectedFilename).to_csv("UninfectedFilename1.csv")

## Getting Histogram Data

In [4]:
#Histogram of all training images
Training = np.zeros((20000, 256))#first 10k will be Parasitized and then Uninfected

#Getting and equalizing histogram
for i in range(10000):#use 10000 files for training
    img = cv2.imread("../DataSet/Parasitized/" + ParasitizedFilename[i], 0)
    equ = cv2.equalizeHist(img)#equalizing histogram
    hist,bins = np.histogram(equ.flatten(),256,[0,256])#change
    Training[i] = hist#filling histogram to Training array
    Training[i][0] = 0

    img = cv2.imread("../DataSet/Uninfected/" + UninfectedFilename[i], 0)
    equ = cv2.equalizeHist(img)#equalizing histogram
    hist,bins = np.histogram(equ.flatten(),256,[0,256])
    Training[i+10000] = hist#filling histogram to Training array
    Training[i+10000][0] = 0

print("Histogram Data has been collected")

Histogram Data has been collected


In [5]:
# #csv of Training Histograms
# pd.DataFrame(Training).to_csv("TrainingHistograms.csv")

## Clusterization

2D - Datapoints List of n d-dimensional vectors. (For this example I already set up 2D Tuples)
Feel free to change to whatever size tuples you want...
<!-- 
# datapoints = [[100,5,1,100], [90,5,1,2], [110,5,1,20], [97,4,1,2], [102,4,1,2], [112,4,1,2], [92,4,1,2], [95,3,1,2], [90,3,1,2], [100,3,1,2],
# [110,5,1,30], [100,5,1,2], [110,4,1,2], [93,3,1,100], [107,2,1,2], [117,3,1,2], [96,2,1,90], [105,3,1,2], [100,3,1,2], [110,3,1,2]] -->

In [6]:
#Euclidian Distance between two d-dimensional points
def eucldist(p0,p1):
    dist = 0.0
    for i in range(0,len(p0)):
        dist += (p0[i] - p1[i])**2
    return math.sqrt(dist)
    
#K-Means Algorithm
def kmeans(k,datapoints):

    # d - Dimensionality of Datapoints
    d = len(datapoints[0]) 
    
    #Limit our iterations
    Max_Iterations = 1000
    i = 0
    
    cluster = [0] * len(datapoints)
    prev_cluster = [-1] * len(datapoints)
    
    #Randomly Choose Centers for the Clusters
    cluster_centers = []
    for i in range(0,k):
        new_cluster = []
        #for i in range(0,d):
        #    new_cluster += [random.randint(0,10)]
        cluster_centers += [random.choice(datapoints)]
        
        
        #Sometimes The Random points are chosen poorly and so there ends up being empty clusters
        #In this particular implementation we want to force K exact clusters.
        #To take this feature off, simply take away "force_recalculation" from the while conditional.
        force_recalculation = False
    
    while (cluster != prev_cluster) or (i > Max_Iterations) or (force_recalculation) :
        
        prev_cluster = list(cluster)
        force_recalculation = False
        i += 1
    
        #Update Point's Cluster Alligiance
        for p in range(0,len(datapoints)):
            min_dist = float("inf")
            
            #Check min_distance against all centers
            for c in range(0,len(cluster_centers)):
                
                dist = eucldist(datapoints[p],cluster_centers[c])
                
                if (dist < min_dist):
                    min_dist = dist  
                    cluster[p] = c   # Reassign Point to new Cluster
        
        
        #Update Cluster's Position
        for k in range(0,len(cluster_centers)):
            new_center = [0] * d
            members = 0
            for p in range(0,len(datapoints)):
                if (cluster[p] == k): #If this point belongs to the cluster
                    for j in range(0,d):
                        new_center[j] += datapoints[p][j]
                    members += 1
            
            for j in range(0,d):
                if members != 0:
                    new_center[j] = new_center[j] / float(members) 
                
                #This means that our initial random assignment was poorly chosen
                #Change it to a new datapoint to actually force k clusters
                else: 
                    new_center = random.choice(datapoints)
                    force_recalculation = True
                    print("Forced Recalculation...")
                    
            
            cluster_centers[k] = new_center
    
        
#     print("======== Results ========")
#     print("Clusters", cluster_centers)
#     print("Iterations",i)
#     print("Assignments", cluster)
    
    return cluster_centers

In [7]:
#Run k-means
centroids = kmeans(k=2, datapoints=Training)

In [8]:
#Another k-means algorithm

# X = Training

# #Euclidian Distance between two d-dimensional points
# def eucldist(p0,p1):
#     dist = 0.0
#     for i in range(0,len(p0)):
#         dist += (p0[i] - p1[i])**2
#     return math.sqrt(dist)

# def Closeness(centroid, relationship):
#     c1_counter = c2_counter = 0
    
#     for i in range(len(X)):
#         if(eucldist(X[i],centroid[0])>eucldist(X[i],centroid[1])):
#             relationship[i] = True
#             c1_counter += 1
#         else:
#             relationship[i] = False
#             c2_counter += 1
#     return centroid, relationship
            
# #finds new centroid
# def findCentroid(relationship):
#     c1_counter = c2_counter = 0
    
#     sumCentroid1 = sumCentroid2 = 0
#     centroids = np.zeros((2))
    
#     for i in range(len(X)):
#         if(relationship[i] == True):
#             sumCentroid1 += X[i]
#             c1_counter += 1
#         else:
#             sumCentroid2 += X[i]
#             c2_counter += 1
            
#     centroids = [[sumCentroid1/c1_counter],[sumCentroid2/c2_counter]]
    
#     return centroids  

# #checks if centroid changed
# def checkChange(centroidsOld, centroidsNew):
#     for i in range(len(X[0])):
#         if(centroidsOld[0][i] == centroidsNew[0][i]):
#             return True
#     return False

# #random centroids
# # centroid = np.zeros((2,2))
# centroid = np.vstack((np.array(X[5]),np.array(X[40])))#change



# relationship = [False]*len(X)
# c1_counter = c2_counter = 0
# sumCentroid1 = sumCentroid2 = 0

# oldCentroid, relationship = Closeness(centroid, relationship)
# newCentroids = findCentroid(relationship)

# # while(checkChange(oldCentroid, newCentroids)):
# while(np.array_equal(np.array(oldCentroid), np.array(newCentroids))):
#     print("Iteration")
#     oldCentroid = newCentroids
#     oldCentroid, relationship = Closeness(oldCentroid, relationship)
#     newCentroids = findCentroid(relationship)
    
# centroids = np.zeros((2, 256))

# centroids[0] = newCentroids[0][0]
# centroids[1] = newCentroids[1][0]

In [9]:
print("centroids have been collected")

centroids have been collected


## Prediction

In [10]:
#Histogram of all TEST images
Test = np.zeros((7558, 256))#remaining 3779 paras and uninfected images

#Getting and equalizing histogram
for i in range(3779):
    img = cv2.imread("../DataSet/Parasitized/" + ParasitizedFilename[i+10000], 0)
    equ = cv2.equalizeHist(img)#equalizing hist
    hist,bins = np.histogram(equ.flatten(),256,[0,256])
    Test[i] = hist
    Test[i][0] = 0

    img = cv2.imread("../DataSet/Uninfected/" + UninfectedFilename[i+10000], 0)
    equ = cv2.equalizeHist(img)#equalizing hist
    hist,bins = np.histogram(equ.flatten(),256,[0,256])
    Test[i+3779] = hist       
    Test[i+3779][0] = 0

In [11]:
TPositive = TNegative = FPositive = FNegative = 0
#TPositive = Actually parasitized and found right
#FPositive = Actually parasitized and found wrong
#TNegative = Actually uninfected and found right
#FNegative = Actually uninfected and found wrong

for i in range (3779):
    
    Distance1 = Distance2 = Distance3 = Distance4 = 0   
    
#     for j in range(256):  #size of dimension
#         Distance1 += (centroids[0][j] - Test[i][j])*(centroids[0][j] - Test[i][j])
#         Distance2 += (centroids[1][j] - Test[i][j])*(centroids[1][j] - Test[i][j])
        
#     for j in range(256):  #size of dimension
#         Distance3 += (centroids[0][j] - Test[i+6890][j])*(centroids[0][j] - Test[i+6890][j])
#         Distance4 += (centroids[1][j] - Test[i+6890][j])*(centroids[1][j] - Test[i+6890][j])
       
    Distance1 = eucldist(p0=centroids[0],p1=Test[i])#Parasitized centroid 0
    Distance2 = eucldist(p0=centroids[1],p1=Test[i])#Parasitized centroid 1
    Distance3 = eucldist(p0=centroids[0],p1=Test[i+3779])#Uninfected centroid 0
    Distance4 = eucldist(p0=centroids[1],p1=Test[i+3779])#Uninfected centroid 1
    
    if(Distance1>Distance2):
        TPositive += 1
    else: 
        FPositive += 1
        
    if(Distance3>Distance4):
        FNegative += 1
    else: 
        TNegative += 1

#If parasitized is another centroid
# if(TPositive<FPositive):
#     TPositive, FPositive = FPositive, TPositive
# if(TNegative<FNegative):
#     TNegative, FNegative = FNegative, TNegative
        
print("TPositive, TNegative, FPositive, FNegative")
print(TPositive, TNegative, FPositive, FNegative)
print("prediction has been competed")

TPositive, TNegative, FPositive, FNegative
457 3313 3322 466
prediction has been competed


## Percentage

In [12]:
print("In parasitized folder = ", TPositive*100/3779, "% right and ", FPositive*100/3779, "% wrong")
print("In uninfected folder = ", TNegative*100/3779, "% right and ", FNegative*100/3779, "% wrong")

In parasitized folder =  12.093146335009262 % right and  87.90685366499073 % wrong
In uninfected folder =  87.66869542206933 % right and  12.33130457793067 % wrong


## Take a specific picture

In [13]:
# #path of image
# # src = "../Parasitized/" + ParasitizedFilename[5]
# # src = "../Uninfected/" + UninfectedFilename[45]
# ParasitizedFilename = os.listdir("/home/mehanton/Documents/Python/DataSet/Parasitized")
# UninfectedFilename = os.listdir("/home/mehanton/Documents/Python/DataSet/Uninfected")

# src = "../DataSet/Parasitized/" 

# for i in range(1000):
#     img = cv2.imread(src + ParasitizedFilename[i], 0)
#     equ = cv2.equalizeHist(img)
#     hist,bins = np.histogram(equ.flatten(),256,[0,256])

#     Distance1 = eucldist(p0=centroids[0],p1=hist)
#     Distance2 = eucldist(p0=centroids[1],p1=hist)

# #     print(Distance1, Distance2)

#     #Centroids could be swaped!
#     if(Distance1>Distance2):
#         print("It's Uninfected")
#     else: 
#         print("It's Parasitized")

## Finish